# Data Preprocessing and Cleaning

This notebook provides two preprocessing approaches:

1. **Approach 1 (with LPR):** Combines AMP and LPR data using lot mapping
2. **Approach 2 (AMP only):** Uses only AMP data which already has zone names

Both approaches include:
- Data consolidation from multiple sheets
- Temporal feature engineering
- Semester-based filtering capabilities
- Data quality checks

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries loaded successfully")

Libraries loaded successfully


## Load Raw Data

In [2]:
# Load Excel file
excel_file = pd.ExcelFile('../data/raw/Data_For_Class_Project.xlsx')

# Load AMP sheets (Automated Metered Parking)
amp1 = pd.read_excel(excel_file, sheet_name='AMP1')
amp2 = pd.read_excel(excel_file, sheet_name='AMP2')
amp3 = pd.read_excel(excel_file, sheet_name='AMP3')

# Load LPR sheets (License Plate Recognition)
lpr_fy23 = pd.read_excel(excel_file, sheet_name='LPR_Reads_FY23')
lpr_fy24 = pd.read_excel(excel_file, sheet_name='LPR_Reads_FY24')
lpr_fy25 = pd.read_excel(excel_file, sheet_name='LPR_Reads_FY25')

# Load lot mapping (force Lot_number as string to preserve leading zeros)
lot_mapping = pd.read_csv('../data/lot_mapping_enhanced.csv', dtype={'Lot_number': str})

print("All data loaded successfully")
print(f"\nAMP data: {len(amp1) + len(amp2) + len(amp3):,} total rows")
print(f"LPR data: {len(lpr_fy23) + len(lpr_fy24) + len(lpr_fy25):,} total rows")
print(f"Lot mappings: {len(lot_mapping)} lots")

All data loaded successfully

AMP data: 1,703,404 total rows
LPR data: 1,780,187 total rows
Lot mappings: 189 lots


## Approach 1: Combine AMP + LPR Data (with lot mapping)

### Step 1: Consolidate and Clean AMP Data

In [3]:
# Consolidate all AMP sheets
amp_combined = pd.concat([amp1, amp2, amp3], ignore_index=True)

print(f"Combined AMP data: {len(amp_combined):,} rows")
print(f"\nColumns: {amp_combined.columns.tolist()}")
print(f"\nDate range: {amp_combined['Start_Date'].min()} to {amp_combined['End_Date'].max()}")

# Check for missing values
print(f"\nMissing values:")
print(amp_combined.isnull().sum())

Combined AMP data: 1,703,404 rows

Columns: ['Zone', 'Start_Date', 'End_Date']

Date range: 2020-08-10 07:51:00 to 2025-11-02 12:38:00

Missing values:
Zone          0
Start_Date    0
End_Date      0
dtype: int64


In [4]:
# Convert date columns to datetime
amp_combined['Start_Date'] = pd.to_datetime(amp_combined['Start_Date'])
amp_combined['End_Date'] = pd.to_datetime(amp_combined['End_Date'])

# Calculate parking duration in hours
amp_combined['duration_hours'] = (amp_combined['End_Date'] - amp_combined['Start_Date']).dt.total_seconds() / 3600

# Remove invalid/outlier durations (e.g., negative or extremely long)
print(f"\nDuration statistics before cleaning:")
print(amp_combined['duration_hours'].describe())

# Filter out unrealistic durations (negative or > 7 days)
amp_clean = amp_combined[
    (amp_combined['duration_hours'] > 0) & 
    (amp_combined['duration_hours'] <= 168)  # 7 days max
].copy()

print(f"\nRows removed: {len(amp_combined) - len(amp_clean):,}")
print(f"Remaining rows: {len(amp_clean):,}")


Duration statistics before cleaning:
count    1.703404e+06
mean     2.123014e+00
std      2.229149e+00
min     -2.400000e+01
25%      1.000000e+00
50%      1.500000e+00
75%      2.483333e+00
max      3.259167e+02
Name: duration_hours, dtype: float64

Rows removed: 537
Remaining rows: 1,702,867


### Step 2: Consolidate and Clean LPR Data

In [5]:
# Consolidate all LPR sheets
lpr_combined = pd.concat([lpr_fy23, lpr_fy24, lpr_fy25], ignore_index=True)

print(f"Combined LPR data: {len(lpr_combined):,} rows")
print(f"\nColumns: {lpr_combined.columns.tolist()}")
print(f"\nDate range: {lpr_combined['Date_Time'].min()} to {lpr_combined['Date_Time'].max()}")

# Check for missing values
print(f"\nMissing values:")
print(lpr_combined.isnull().sum())

Combined LPR data: 1,780,187 rows

Columns: ['Date_Time', 'LOT']

Date range: 2022-07-01 05:24:26 to 2025-06-30 21:58:49

Missing values:
Date_Time    0
LOT          0
dtype: int64


In [6]:
# Convert date column to datetime
lpr_combined['Date_Time'] = pd.to_datetime(lpr_combined['Date_Time'])

# Extract lot number from 'LOT XXX' format (take only first 3 digits/characters)
# This handles cases like "LOT 020 HOURLY" → "020"
lpr_combined['Lot_number'] = lpr_combined['LOT'].str.replace('LOT ', '').str.strip().str[:3]

# Remove rows with missing lot numbers
lpr_clean = lpr_combined.dropna(subset=['Lot_number']).copy()

print(f"\nRows removed: {len(lpr_combined) - len(lpr_clean):,}")
print(f"Remaining rows: {len(lpr_clean):,}")
print(f"Unique lots: {lpr_clean['Lot_number'].nunique()}")


Rows removed: 0
Remaining rows: 1,780,187
Unique lots: 185


### Step 3: Merge LPR Data with Lot Mapping

In [7]:
# Ensure lot_number format is consistent
lot_mapping['Lot_number'] = lot_mapping['Lot_number'].astype(str).str.strip()
lpr_clean['Lot_number'] = lpr_clean['Lot_number'].astype(str).str.strip()

# Merge LPR data with lot mapping
lpr_mapped = lpr_clean.merge(
    lot_mapping[['Lot_number', 'Zone_Name', 'zone_type', 'location_description']], 
    on='Lot_number', 
    how='left'
)

# Check mapping coverage
mapped_count = lpr_mapped['Zone_Name'].notna().sum()
total_count = len(lpr_mapped)
coverage_pct = (mapped_count / total_count * 100)

print(f"Mapping coverage:")
print(f"  Mapped records: {mapped_count:,} ({coverage_pct:.1f}%)")
print(f"  Unmapped records: {total_count - mapped_count:,} ({100-coverage_pct:.1f}%)")

# Show unmapped lots
unmapped_lots = lpr_mapped[lpr_mapped['Zone_Name'].isna()]['Lot_number'].unique()
print(f"\nUnmapped lot numbers ({len(unmapped_lots)}):")
print(sorted(unmapped_lots)[:20])  # Show first 20

Mapping coverage:
  Mapped records: 1,780,207 (100.0%)
  Unmapped records: 184 (0.0%)

Unmapped lot numbers (2):
['021', '101']


### Step 4: Add Temporal Features to Both Datasets

In [8]:
def add_temporal_features(df, datetime_col):
    """
    Add temporal features for time-based analysis and filtering.
    
    Parameters:
    df: DataFrame with datetime column
    datetime_col: Name of the datetime column
    """
    df = df.copy()
    
    # Basic time features
    df['year'] = df[datetime_col].dt.year
    df['month'] = df[datetime_col].dt.month
    df['day'] = df[datetime_col].dt.day
    df['hour'] = df[datetime_col].dt.hour
    df['day_of_week'] = df[datetime_col].dt.dayofweek  # 0=Monday, 6=Sunday
    df['day_name'] = df[datetime_col].dt.day_name()
    df['week_of_year'] = df[datetime_col].dt.isocalendar().week
    
    # Is weekend?
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    
    # Time of day categories
    df['time_of_day'] = pd.cut(
        df['hour'],
        bins=[-1, 6, 12, 17, 21, 24],
        labels=['Night', 'Morning', 'Afternoon', 'Evening', 'Late Night']
    )
    
    return df

# Add temporal features to AMP data
amp_clean = add_temporal_features(amp_clean, 'Start_Date')

# Add temporal features to LPR data
lpr_mapped = add_temporal_features(lpr_mapped, 'Date_Time')

print("Temporal features added successfully")
print(f"\nNew columns in AMP data: {amp_clean.columns.tolist()[-10:]}")
print(f"New columns in LPR data: {lpr_mapped.columns.tolist()[-10:]}")

Temporal features added successfully

New columns in AMP data: ['duration_hours', 'year', 'month', 'day', 'hour', 'day_of_week', 'day_name', 'week_of_year', 'is_weekend', 'time_of_day']
New columns in LPR data: ['location_description', 'year', 'month', 'day', 'hour', 'day_of_week', 'day_name', 'week_of_year', 'is_weekend', 'time_of_day']


### Step 5: Add Semester Labels

In [9]:
def assign_semester(date):
    """
    Assign academic semester based on date.
    
    WSU academic calendar approximation:
    - Fall: August - December
    - Spring: January - May
    - Summer: June - July
    """
    month = date.month
    year = date.year
    
    if month >= 8:  # August onwards = Fall
        return f"Fall {year}"
    elif month <= 5:  # January to May = Spring
        return f"Spring {year}"
    else:  # June-July = Summer
        return f"Summer {year}"

# Add semester to AMP data
amp_clean['semester'] = amp_clean['Start_Date'].apply(assign_semester)

# Add semester to LPR data
lpr_mapped['semester'] = lpr_mapped['Date_Time'].apply(assign_semester)

# Show semester distribution
print("AMP Data - Semester Distribution:")
print(amp_clean['semester'].value_counts().sort_index())

print("\nLPR Data - Semester Distribution:")
print(lpr_mapped['semester'].value_counts().sort_index())

AMP Data - Semester Distribution:
semester
Fall 2020       10518
Fall 2021       76206
Fall 2022      138616
Fall 2023      219145
Fall 2024      221559
Fall 2025      138565
Spring 2021     20202
Spring 2022    109230
Spring 2023    203690
Spring 2024    235645
Spring 2025    240084
Summer 2021      7184
Summer 2022     12591
Summer 2023     20971
Summer 2024     24688
Summer 2025     23973
Name: count, dtype: int64

LPR Data - Semester Distribution:
semester
Fall 2022      165863
Fall 2023      323961
Fall 2024      312261
Spring 2023    314784
Spring 2024    258281
Spring 2025    271918
Summer 2022     20571
Summer 2023     25893
Summer 2024     54947
Summer 2025     31912
Name: count, dtype: int64


### Step 6: Save Preprocessed Data (Approach 1)

In [10]:
# Save preprocessed data
amp_clean.to_csv('../data/processed/amp_preprocessed.csv', index=False)
lpr_mapped.to_csv('../data/processed/lpr_preprocessed.csv', index=False)

print("Approach 1 data saved:")
print(f"  ../data/processed/amp_preprocessed.csv ({len(amp_clean):,} rows)")
print(f"  ../data/processed/lpr_preprocessed.csv ({len(lpr_mapped):,} rows)")

Approach 1 data saved:
  ../data/processed/amp_preprocessed.csv (1,702,867 rows)
  ../data/processed/lpr_preprocessed.csv (1,780,391 rows)


## Approach 2: AMP Data Only (without LPR)

### Process AMP Data with Enhanced Features

In [11]:
# Start with the already cleaned AMP data
amp_only = amp_clean.copy()

# Add zone type classification based on Zone_Name
def classify_zone_type(zone_name):
    """
    Classify zone type based on zone name.
    """
    zone_name = str(zone_name).lower()
    
    if any(x in zone_name for x in ['orange', 'garage']):
        return 'Paid'
    elif any(x in zone_name for x in ['gray', 'crimson', 'resident']):
        return 'Resident'
    else:
        return 'Permit'

amp_only['zone_type'] = amp_only['Zone'].apply(classify_zone_type)

print("Zone type distribution:")
print(amp_only['zone_type'].value_counts())

# Show sample of unique zones
print("\nUnique zones in data:")
print(amp_only['Zone'].value_counts())

Zone type distribution:
zone_type
Permit    1009019
Paid       693848
Name: count, dtype: int64

Unique zones in data:
Zone
CUE Garage                                                        399151
Library Garage                                                    291173
Student Rec Center                                                247802
Columbia Street Lot Top Bays                                      135279
Green 5 South Beasley                                             104820
Wilson Road on Street Meters                                       88960
Cougar Way on Street Meters                                        50527
Ferdinand's Ice Cream Shoppe Parking                               35275
Green 1 PACCAR South                                               30520
Green 3: (Gravel lot) College/Spokane                              27784
Green 2 KMac Lot                                                   27357
Green 3: Washington St street-side & Spark                         24397


### Create Hourly Parking Events from Sessions

In [12]:
# For each parking session, create hourly records
# This helps track occupancy over time

def expand_session_to_hours(row):
    """
    Expand a parking session into hourly records.
    Each hour the vehicle was parked creates one record.
    """
    start = row['Start_Date']
    end = row['End_Date']
    
    # Create hourly range
    hours = pd.date_range(start=start.floor('h'), end=end.floor('h'), freq='h')
    
    return hours

# Sample first 1000 rows for demonstration (processing all rows may take time)
sample_amp = amp_only.head(1000).copy()

# Expand to hourly records
hourly_records = []
for idx, row in sample_amp.iterrows():
    hours = expand_session_to_hours(row)
    for hour in hours:
        hourly_records.append({
            'datetime': hour,
            'zone': row['Zone'],
            'zone_type': row['zone_type'],
            'semester': row['semester']
        })

hourly_df = pd.DataFrame(hourly_records)

print(f"Expanded {len(sample_amp):,} sessions into {len(hourly_df):,} hourly records")
print("\nSample of hourly data:")
print(hourly_df.head(10))

Expanded 1,000 sessions into 3,422 hourly records

Sample of hourly data:
             datetime                          zone zone_type   semester
0 2020-08-10 07:00:00   Cougar Way on Street Meters    Permit  Fall 2020
1 2020-08-10 08:00:00   Cougar Way on Street Meters    Permit  Fall 2020
2 2020-08-10 08:00:00  Thatuna Rd. on Street Meters    Permit  Fall 2020
3 2020-08-10 09:00:00  Thatuna Rd. on Street Meters    Permit  Fall 2020
4 2020-08-10 08:00:00                Library Garage      Paid  Fall 2020
5 2020-08-10 09:00:00                Library Garage      Paid  Fall 2020
6 2020-08-10 08:00:00          Green 1 PACCAR South    Permit  Fall 2020
7 2020-08-10 09:00:00          Green 1 PACCAR South    Permit  Fall 2020
8 2020-08-10 09:00:00                Library Garage      Paid  Fall 2020
9 2020-08-18 11:00:00                    CUE Garage      Paid  Fall 2020


### Calculate Hourly Occupancy per Zone

In [13]:
# Count vehicles per hour per zone
occupancy = hourly_df.groupby(['datetime', 'zone', 'zone_type', 'semester']).size().reset_index(name='vehicle_count')

print("Hourly occupancy data:")
print(occupancy.head(20))

print("\nOccupancy statistics by zone:")
print(occupancy.groupby('zone')['vehicle_count'].describe())

Hourly occupancy data:
              datetime                                   zone zone_type  \
0  2020-08-10 07:00:00            Cougar Way on Street Meters    Permit   
1  2020-08-10 08:00:00            Cougar Way on Street Meters    Permit   
2  2020-08-10 08:00:00                   Green 1 PACCAR South    Permit   
3  2020-08-10 08:00:00                         Library Garage      Paid   
4  2020-08-10 08:00:00           Thatuna Rd. on Street Meters    Permit   
5  2020-08-10 09:00:00                   Green 1 PACCAR South    Permit   
6  2020-08-10 09:00:00                         Library Garage      Paid   
7  2020-08-10 09:00:00           Thatuna Rd. on Street Meters    Permit   
8  2020-08-18 11:00:00                             CUE Garage      Paid   
9  2020-08-18 12:00:00                             CUE Garage      Paid   
10 2020-08-18 12:00:00  Green 3: College Avenue -or- Idaho St    Permit   
11 2020-08-18 12:00:00           Wilson Road on Street Meters    Permit   
12

### Save Preprocessed Data (Approach 2)

In [14]:
# Save AMP-only preprocessed data
amp_only.to_csv('../data/processed/amp_only_preprocessed.csv', index=False)

# Save sample hourly occupancy (for demonstration)
occupancy.to_csv('../data/processed/hourly_occupancy_sample.csv', index=False)

print("Approach 2 data saved:")
print(f"  ../data/processed/amp_only_preprocessed.csv ({len(amp_only):,} rows)")
print(f"  ../data/processed/hourly_occupancy_sample.csv ({len(occupancy):,} rows)")

Approach 2 data saved:
  ../data/processed/amp_only_preprocessed.csv (1,702,867 rows)
  ../data/processed/hourly_occupancy_sample.csv (1,525 rows)


## Data Quality Summary

In [15]:
print("="*60)
print("DATA PREPROCESSING SUMMARY")
print("="*60)

print("\nAPPROACH 1: AMP + LPR (with lot mapping)")
print(f"  AMP records: {len(amp_clean):,}")
print(f"  LPR records: {len(lpr_mapped):,}")
print(f"  LPR mapping coverage: {coverage_pct:.1f}%")
print(f"  Date range: {amp_clean['Start_Date'].min().date()} to {amp_clean['End_Date'].max().date()}")
print(f"  Semesters covered: {amp_clean['semester'].nunique()}")

print("\nAPPROACH 2: AMP only")
print(f"  AMP records: {len(amp_only):,}")
print(f"  Unique zones: {amp_only['Zone'].nunique()}")
print(f"  Zone types: {amp_only['zone_type'].value_counts().to_dict()}")
print(f"  Date range: {amp_only['Start_Date'].min().date()} to {amp_only['End_Date'].max().date()}")



DATA PREPROCESSING SUMMARY

APPROACH 1: AMP + LPR (with lot mapping)
  AMP records: 1,702,867
  LPR records: 1,780,391
  LPR mapping coverage: 100.0%
  Date range: 2020-08-10 to 2025-11-02
  Semesters covered: 16

APPROACH 2: AMP only
  AMP records: 1,702,867
  Unique zones: 63
  Zone types: {'Permit': 1009019, 'Paid': 693848}
  Date range: 2020-08-10 to 2025-11-02


## Semester-Based Filtering Example

In [16]:
# Example: Filter data for Fall 2023 semester
fall_2023_amp = amp_clean[amp_clean['semester'] == 'Fall 2023']
fall_2023_lpr = lpr_mapped[lpr_mapped['semester'] == 'Fall 2023']

print(f"Fall 2023 AMP records: {len(fall_2023_amp):,}")
print(f"Fall 2023 LPR records: {len(fall_2023_lpr):,}")

# Example: Filter for weekdays only
weekday_amp = amp_clean[amp_clean['is_weekend'] == 0]
print(f"\nWeekday AMP records: {len(weekday_amp):,}")

# Example: Filter for peak hours (8 AM - 5 PM)
peak_hours_amp = amp_clean[(amp_clean['hour'] >= 8) & (amp_clean['hour'] <= 17)]
print(f"Peak hours AMP records: {len(peak_hours_amp):,}")

# Example: Combine filters (Fall 2023, weekdays, peak hours)
filtered = amp_clean[
    (amp_clean['semester'] == 'Fall 2023') & 
    (amp_clean['is_weekend'] == 0) &
    (amp_clean['hour'] >= 8) & 
    (amp_clean['hour'] <= 17)
]
print(f"\nFall 2023 weekday peak hours: {len(filtered):,} records")

Fall 2023 AMP records: 219,145
Fall 2023 LPR records: 323,961

Weekday AMP records: 1,633,676
Peak hours AMP records: 1,482,363

Fall 2023 weekday peak hours: 180,024 records


## Complete Semester-Based Filtering (2020-2025)

Comprehensive filtering examples for all available semesters in the dataset

In [17]:
# Create comprehensive semester filtering summary for ALL years
print("="*50)
print("COMPREHENSIVE SEMESTER FILTERING GUIDE (2020-2025)")
print("="*50)

# Get all unique semesters
all_semesters = sorted(amp_clean['semester'].unique())

print(f"\nTotal semesters available: {len(all_semesters)}")
print("\nALL AVAILABLE SEMESTERS:")
for sem in all_semesters:
    amp_count = len(amp_clean[amp_clean['semester'] == sem])
    lpr_count = len(lpr_mapped[lpr_mapped['semester'] == sem])
    print(f"  {sem:15} | AMP: {amp_count:>8,} records | LPR: {lpr_count:>8,} records")

# Show filtering examples for each year
print("\n" + "="*50)
print("FILTERING EXAMPLES BY YEAR")
print("="*50)

for year in [2020, 2021, 2022, 2023, 2024, 2025]:
    print(f"\n### YEAR {year} ###")
    
    # Get all semesters for this year
    year_semesters = [s for s in all_semesters if str(year) in s]
    
    if not year_semesters:
        print(f"  No data available for {year}")
        continue
    
    # Show each semester for this year
    for semester in year_semesters:
        semester_data = amp_clean[amp_clean['semester'] == semester]
        
        # Basic filter
        print(f"\n{semester}:")
        print(f"  All records: {len(semester_data):,}")
        
        # Weekday filter
        weekday_data = semester_data[semester_data['is_weekend'] == 0]
        print(f"  Weekdays only: {len(weekday_data):,}")
        
        # Peak hours filter
        peak_data = semester_data[(semester_data['hour'] >= 8) & (semester_data['hour'] <= 17)]
        print(f"  Peak hours (8AM-5PM): {len(peak_data):,}")
        
        # Combined filter: weekdays + peak hours
        combined = semester_data[
            (semester_data['is_weekend'] == 0) &
            (semester_data['hour'] >= 8) &
            (semester_data['hour'] <= 17)
        ]
        print(f"  Weekdays + Peak hours: {len(combined):,}")
        
        # Show date range
        print(f"  Date range: {semester_data['Start_Date'].min().date()} to {semester_data['Start_Date'].max().date()}")

COMPREHENSIVE SEMESTER FILTERING GUIDE (2020-2025)

Total semesters available: 16

ALL AVAILABLE SEMESTERS:
  Fall 2020       | AMP:   10,518 records | LPR:        0 records
  Fall 2021       | AMP:   76,206 records | LPR:        0 records
  Fall 2022       | AMP:  138,616 records | LPR:  165,863 records
  Fall 2023       | AMP:  219,145 records | LPR:  323,961 records
  Fall 2024       | AMP:  221,559 records | LPR:  312,261 records
  Fall 2025       | AMP:  138,565 records | LPR:        0 records
  Spring 2021     | AMP:   20,202 records | LPR:        0 records
  Spring 2022     | AMP:  109,230 records | LPR:        0 records
  Spring 2023     | AMP:  203,690 records | LPR:  314,784 records
  Spring 2024     | AMP:  235,645 records | LPR:  258,281 records
  Spring 2025     | AMP:  240,084 records | LPR:  271,918 records
  Summer 2021     | AMP:    7,184 records | LPR:        0 records
  Summer 2022     | AMP:   12,591 records | LPR:   20,571 records
  Summer 2023     | AMP:   20,971 